In [1]:
# coding: utf-8
import numpy as np
import pandas as pd
from pandas_datareader import data as web
#import tensorflow as tf
import chainer.functions  as F
from chainer import FunctionSet
from chainer import Variable
from chainer import optimizers
#from chainer import cuda
from datetime import datetime
#from sklearn.datasets import fetch_mldata
import model_util
models=model_util.ModelUtil()

In [2]:
def set_data(start, end):
    # ヒストリカルデータをダウンロードする。
    snp = web.DataReader('^GSPC', 'yahoo', start, end)
    nyse = web.DataReader('^NYA', 'yahoo', start, end)
    djia = web.DataReader('^DJI', 'yahoo', start, end)
    nikkei = web.DataReader('^N225', 'yahoo', start, end)
    hangseng = web.DataReader('000001.SS', 'yahoo', start, end)
    ftse = web.DataReader('ISF.L', 'yahoo', start, end)  # ETFで代用。
    dax = web.DataReader('^GDAXI', 'yahoo', start, end)
    aord = web.DataReader('^AORD', 'yahoo', start, end)

    # 終値を格納する。
    closing_data = pd.DataFrame()
    closing_data['snp_close'] = snp['Close']
    closing_data['nyse_close'] = nyse['Close']
    closing_data['djia_close'] = djia['Close']
    closing_data['nikkei_close'] = nikkei['Close']
    closing_data['hangseng_close'] = hangseng['Close']
    closing_data['ftse_close'] = ftse['Close']
    closing_data['dax_close'] = dax['Close']
    closing_data['aord_close'] = aord['Close']

    # 終値の欠損値を前のデータで補間する。
    closing_data = closing_data.fillna(method='ffill')

    # 終値の対数変化率を格納する。
    log_return_data = pd.DataFrame()
    log_return_data['snp_log_return'] = (np.log(closing_data['snp_close'] /
        closing_data['snp_close'].shift()))
    log_return_data['nyse_log_return'] = (np.log(closing_data['nyse_close'] /
        closing_data['nyse_close'].shift()))
    log_return_data['djia_log_return'] = (np.log(closing_data['djia_close'] /
        closing_data['djia_close'].shift()))
    log_return_data['nikkei_log_return'] = (np.log(closing_data['nikkei_close']
        / closing_data['nikkei_close'].shift()))
    log_return_data['hangseng_log_return'] = (
        np.log(closing_data['hangseng_close'] /
        closing_data['hangseng_close'].shift()))
    log_return_data['ftse_log_return'] = (np.log(closing_data['ftse_close'] /
        closing_data['ftse_close'].shift()))
    log_return_data['dax_log_return'] = (np.log(closing_data['dax_close'] /
        closing_data['dax_close'].shift()))
    log_return_data['aord_log_return'] = (np.log(closing_data['aord_close'] /
        closing_data['aord_close'].shift()))

    # S&P500の対数変化率が0以上なら1、さもなければ0を格納した列を加える。
    log_return_data['snp_log_return_positive'] = 0
    log_return_data.ix[log_return_data['snp_log_return'] >= 0,
                       'snp_log_return_positive'] = 1

    # S&P500の対数変化率が0未満なら1、さもなければ0を格納した列を加える。
    log_return_data['snp_log_return_negative'] = 0
    log_return_data.ix[log_return_data['snp_log_return'] < 0,
                       'snp_log_return_negative'] = 1

    # 学習・テスト用データを作成する。
    training_test_data = pd.DataFrame(
        columns=[
            'snp_log_return_positive', 'snp_log_return_negative',
            'snp_log_return_1', 'snp_log_return_2', 'snp_log_return_3',
            'nyse_log_return_1', 'nyse_log_return_2', 'nyse_log_return_3',
            'djia_log_return_1', 'djia_log_return_2', 'djia_log_return_3',
            'nikkei_log_return_1', 'nikkei_log_return_2', 'nikkei_log_return_3',
            'hangseng_log_return_1', 'hangseng_log_return_2',
                'hangseng_log_return_3',
            'ftse_log_return_1', 'ftse_log_return_2', 'ftse_log_return_3',
            'dax_log_return_1', 'dax_log_return_2', 'dax_log_return_3',
            'aord_log_return_1', 'aord_log_return_2', 'aord_log_return_3'])
    for i in range(7, len(log_return_data)):
        snp_log_return_positive = (
            log_return_data['snp_log_return_positive'].ix[i])
        snp_log_return_negative = (
            log_return_data['snp_log_return_negative'].ix[i])

        # 先読みバイアスを排除するため、当日のデータを使わない。
        snp_log_return_1 = log_return_data['snp_log_return'].ix[i]
        snp_log_return_2 = log_return_data['snp_log_return'].ix[i-1]
        snp_log_return_3 = log_return_data['snp_log_return'].ix[i-2]
        nyse_log_return_1 = log_return_data['nyse_log_return'].ix[i]
        nyse_log_return_2 = log_return_data['nyse_log_return'].ix[i-1]
        nyse_log_return_3 = log_return_data['nyse_log_return'].ix[i-2]
        djia_log_return_1 = log_return_data['djia_log_return'].ix[i]
        djia_log_return_2 = log_return_data['djia_log_return'].ix[i-1]
        djia_log_return_3 = log_return_data['djia_log_return'].ix[i-2]
        nikkei_log_return_1 = log_return_data['nikkei_log_return'].ix[i]
        nikkei_log_return_2 = log_return_data['nikkei_log_return'].ix[i-1]
        nikkei_log_return_3 = log_return_data['nikkei_log_return'].ix[i-2]
        hangseng_log_return_1 = log_return_data['hangseng_log_return'].ix[i]
        hangseng_log_return_2 = log_return_data['hangseng_log_return'].ix[i-1]
        hangseng_log_return_3 = log_return_data['hangseng_log_return'].ix[i-2]
        ftse_log_return_1 = log_return_data['ftse_log_return'].ix[i]
        ftse_log_return_2 = log_return_data['ftse_log_return'].ix[i-1]
        ftse_log_return_3 = log_return_data['ftse_log_return'].ix[i-2]
        dax_log_return_1 = log_return_data['dax_log_return'].ix[i]
        dax_log_return_2 = log_return_data['dax_log_return'].ix[i-1]
        dax_log_return_3 = log_return_data['dax_log_return'].ix[i-2]
        aord_log_return_1 = log_return_data['aord_log_return'].ix[i]
        aord_log_return_2 = log_return_data['aord_log_return'].ix[i-1]
        aord_log_return_3 = log_return_data['aord_log_return'].ix[i-2]

        # 各データをインデックスのラベルを使用しないで結合する。
        training_test_data = training_test_data.append(
            {'snp_log_return_positive':snp_log_return_positive,
            'snp_log_return_negative':snp_log_return_negative,
            'snp_log_return_1':snp_log_return_1,
            'snp_log_return_2':snp_log_return_2,
            'snp_log_return_3':snp_log_return_3,
            'nyse_log_return_1':nyse_log_return_1,
            'nyse_log_return_2':nyse_log_return_2,
            'nyse_log_return_3':nyse_log_return_3,
            'djia_log_return_1':djia_log_return_1,
            'djia_log_return_2':djia_log_return_2,
            'djia_log_return_3':djia_log_return_3,
            'nikkei_log_return_1':nikkei_log_return_1,
            'nikkei_log_return_2':nikkei_log_return_2,
            'nikkei_log_return_3':nikkei_log_return_3,
            'hangseng_log_return_1':hangseng_log_return_1,
            'hangseng_log_return_2':hangseng_log_return_2,
            'hangseng_log_return_3':hangseng_log_return_3,
            'ftse_log_return_1':ftse_log_return_1,
            'ftse_log_return_2':ftse_log_return_2,
            'ftse_log_return_3':ftse_log_return_3,
            'dax_log_return_1':dax_log_return_1,
            'dax_log_return_2':dax_log_return_2,
            'dax_log_return_3':dax_log_return_3,
            'aord_log_return_1':aord_log_return_1,
            'aord_log_return_2':aord_log_return_2,
            'aord_log_return_3':aord_log_return_3},
            ignore_index=True)

    # 3列目以降を説明変数として格納する。
    predictors_tf = training_test_data[training_test_data.columns[2:]]

    # 1、2列目を目的変数として格納する。
    classes_tf = training_test_data[training_test_data.columns[:2]]

    # 学習用セットのサイズを学習・テスト用データの80%に設定する。
    training_set_size = int(len(training_test_data) * 0.8)

    # 説明変数の初めの80%を学習用データにする。
    x_train = predictors_tf[:training_set_size].as_matrix()
    
    # 目的変数の初めの80%を学習用データにする。
    y_train = classes_tf[:training_set_size].as_matrix()

    # 説明変数の残りの20%をテスト用データにする。
    x_test = predictors_tf[training_set_size:].as_matrix()

    # 目的変数の残りの20%をテスト用データにする。
    y_test = classes_tf[training_set_size:].as_matrix()
    
    return x_train, x_test, y_train, y_test, training_set_size

In [3]:
# ニューラルネットの構造
def forward(x, train=True):
    # 活性化関数を選択
    # sigmoid関数
#     h1 = F.sigmoid(model.l1(x))
#     h2 = F.sigmoid(model.l2(h1))

    # dropoutを追加
    # ReLU関数
    h1 = F.dropout(F.relu(model.l1(x)),  train=train)
    h2 = F.dropout(F.relu(model.l2(h1)), train=train)
    y  = model.l3(h2)

    return y

In [4]:
def train(x_batch, y_batch, train=True):
    # 型を変換
    x = Variable(x_batch.astype(np.float32), volatile=False) 
    t = Variable(y_batch.astype(np.float32), volatile=False)
    p = forward(x, train)

    # 勾配を初期化
    optimizer.zero_grads()
#     print("x: ndim {0} data {1}".format(x.ndim,x.data))
#     print("p: ndim {0} data {1}".format(p.ndim,p.data))
#     print("p_class: {}".format(p.data.argmax()))

    # 順伝播させて誤差と精度を算出
    # 多クラス分類なので誤差関数としてソフトマックス関数の
    # 交差エントロピー関数を用いて、誤差を導出
#     loss = F.softmax_cross_entropy(p, t)
    loss = F.mean_squared_error(p, t)
#     acc = F.accuracy(p, t)

    # 誤差逆伝播で勾配を計算
    loss.backward()
    optimizer.update()
#     return loss, acc
    return loss

In [5]:
# ヒストリカルデータの開始日と終了日を設定する。
start = datetime(2010, 1, 1)
end = datetime(2016, 11, 23)

x_train, x_test, y_train, y_test, N = set_data(start,end)

N_test = len(y_test)

In [6]:
# ターゲットモデル名
target='kabu_price'

# Prepare multi-layer perceptron model
# 多層パーセプトロンモデルの設定
# 入力 784次元、出力 10次元
# 中間層のunit数
# 一層目
n_units = 120
# 二層目
n2_units = 240

model = FunctionSet(l1=F.Linear(24, n_units),
                    l2=F.Linear(n_units, n2_units),
                    l3=F.Linear(n2_units, 2))
model.compute_accuracy = False #accuracyを計算しない

# Setup optimizer
optimizer = optimizers.Adam()
optimizer.setup(model)

train_loss = []
train_acc  = []
test_loss = []
test_acc  = []

l1_W = []
l2_W = []
l3_W = []

# 確率的勾配降下法で学習させる際の１回分のバッチサイズ
batchsize = 4

# 学習の繰り返し回数
n_epoch = 20
#n_epoch   = 5

In [7]:
# Learning loop
for epoch in xrange(1, n_epoch+1):
    print 'epoch', epoch

    # training
    # N個の順番をランダムに並び替える
    perm = np.random.permutation(N)
    sum_accuracy = 0
    sum_loss = 0

    # 0〜Nまでのデータをバッチサイズごとに使って学習
    for i in xrange(0, N, batchsize):
        x_batch = x_train[perm[i:i+batchsize]]
        y_batch = y_train[perm[i:i+batchsize]]
#         loss, acc = train(x_batch, y_batch)
        loss = train(x_batch, y_batch)

#         sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
#         sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize
        sum_loss     += float(loss.data) * batchsize
#         sum_accuracy += float(acc.data) * batchsize

    # 訓練データの誤差と、正解精度を表示
    #print 'train mean loss={}, accuracy={}'.format(sum_loss / N, sum_accuracy / N)

    train_loss.append(sum_loss / N)
#     train_acc.append(sum_accuracy / N)

    # evaluation
    # テストデータで誤差と、正解精度を算出し汎化性能を確認
#     sum_accuracy = 0
    sum_loss     = 0
    for i in xrange(0, N_test, batchsize):
        x_batch = x_test[i:i+batchsize]
        y_batch = y_test[i:i+batchsize]
#         loss, acc = train(x_batch, y_batch)
        loss = train(x_batch, y_batch)

#         sum_loss     += float(cuda.to_cpu(loss.data)) * batchsize
#         sum_accuracy += float(cuda.to_cpu(acc.data)) * batchsize
        sum_loss     += float(loss.data) * batchsize
#         sum_accuracy += float(acc.data) * batchsize

    # テストデータでの誤差と、正解精度を表示
    # print 'test  mean loss={}'.format(sum_loss / N_test)
#     print 'test  mean loss={}, accuracy={}'.format(sum_loss / N_test, sum_accuracy / N_test)
    test_loss.append(sum_loss / N_test)
#     test_acc.append(sum_accuracy / N_test)

    # 学習したパラメーターを保存
    l1_W.append(model.l1.W)
    l2_W.append(model.l2.W)
    l3_W.append(model.l3.W)
    
models.set_model_pkl('{0}.pkl'.format(target))
models.set_model_name('{0}.model'.format(target))
models.set_optimizer_name('{0}.state'.format(target))
models.dump_model(model)
models.dump_model_and_optimizer(model, optimizer)
    
# 精度と誤差をグラフ描画
# plt.figure(figsize=(8,6))
# plt.plot(range(len(train_acc)), train_acc)
# plt.plot(range(len(test_acc)), test_acc)
# plt.legend(["train_acc","test_acc"],loc=4)
# plt.title("Accuracy of digit recognition.")
# plt.plot()

epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
save the model
save the optimizer


In [136]:
def set_production_data(start, end):
    # ヒストリカルデータをダウンロードする。
    snp = web.DataReader('^GSPC', 'yahoo', start, end)
    nyse = web.DataReader('^NYA', 'yahoo', start, end)
    djia = web.DataReader('^DJI', 'yahoo', start, end)
    nikkei = web.DataReader('^N225', 'yahoo', start, end)
    hangseng = web.DataReader('000001.SS', 'yahoo', start, end)
    ftse = web.DataReader('ISF.L', 'yahoo', start, end)  # ETFで代用。
    dax = web.DataReader('^GDAXI', 'yahoo', start, end)
    aord = web.DataReader('^AORD', 'yahoo', start, end)

    # label格納
    label_data = nikkei.index
    
    # 終値を格納する。
    closing_data = pd.DataFrame()
    closing_data['snp_close'] = snp['Close']
    closing_data['nyse_close'] = nyse['Close']
    closing_data['djia_close'] = djia['Close']
    closing_data['nikkei_close'] = nikkei['Close']
    closing_data['hangseng_close'] = hangseng['Close']
    closing_data['ftse_close'] = ftse['Close']
    closing_data['dax_close'] = dax['Close']
    closing_data['aord_close'] = aord['Close']

    # 終値の欠損値を前のデータで補間する。
    closing_data = closing_data.fillna(method='ffill')

    # 終値の対数変化率を格納する。
    log_return_data = pd.DataFrame()
    log_return_data['snp_log_return'] = (np.log(closing_data['snp_close'] /
        closing_data['snp_close'].shift()))
    log_return_data['nyse_log_return'] = (np.log(closing_data['nyse_close'] /
        closing_data['nyse_close'].shift()))
    log_return_data['djia_log_return'] = (np.log(closing_data['djia_close'] /
        closing_data['djia_close'].shift()))
    log_return_data['nikkei_log_return'] = (np.log(closing_data['nikkei_close']
        / closing_data['nikkei_close'].shift()))
    log_return_data['hangseng_log_return'] = (
        np.log(closing_data['hangseng_close'] /
        closing_data['hangseng_close'].shift()))
    log_return_data['ftse_log_return'] = (np.log(closing_data['ftse_close'] /
        closing_data['ftse_close'].shift()))
    log_return_data['dax_log_return'] = (np.log(closing_data['dax_close'] /
        closing_data['dax_close'].shift()))
    log_return_data['aord_log_return'] = (np.log(closing_data['aord_close'] /
        closing_data['aord_close'].shift()))

    # S&P500の対数変化率が0以上なら1、さもなければ0を格納した列を加える。
    log_return_data['snp_log_return_positive'] = 0
    log_return_data.ix[log_return_data['snp_log_return'] >= 0,
                       'snp_log_return_positive'] = 1

    # S&P500の対数変化率が0未満なら1、さもなければ0を格納した列を加える。
    log_return_data['snp_log_return_negative'] = 0
    log_return_data.ix[log_return_data['snp_log_return'] < 0,
                       'snp_log_return_negative'] = 1

    # 学習・テスト用データを作成する。
    training_test_data = pd.DataFrame(
        columns=[
            'snp_log_return_positive', 'snp_log_return_negative',
            'snp_log_return_1', 'snp_log_return_2', 'snp_log_return_3',
            'nyse_log_return_1', 'nyse_log_return_2', 'nyse_log_return_3',
            'djia_log_return_1', 'djia_log_return_2', 'djia_log_return_3',
            'nikkei_log_return_1', 'nikkei_log_return_2', 'nikkei_log_return_3',
            'hangseng_log_return_1', 'hangseng_log_return_2',
                'hangseng_log_return_3',
            'ftse_log_return_1', 'ftse_log_return_2', 'ftse_log_return_3',
            'dax_log_return_1', 'dax_log_return_2', 'dax_log_return_3',
            'aord_log_return_1', 'aord_log_return_2', 'aord_log_return_3'])

    for i in range(7, len(log_return_data)):
        snp_log_return_positive = (
            log_return_data['snp_log_return_positive'].ix[i])
        snp_log_return_negative = (
            log_return_data['snp_log_return_negative'].ix[i])

        # 先読みバイアスを排除するため、当日のデータを使わない。
        snp_log_return_1 = log_return_data['snp_log_return'].ix[i]
        snp_log_return_2 = log_return_data['snp_log_return'].ix[i-1]
        snp_log_return_3 = log_return_data['snp_log_return'].ix[i-2]
        nyse_log_return_1 = log_return_data['nyse_log_return'].ix[i]
        nyse_log_return_2 = log_return_data['nyse_log_return'].ix[i-1]
        nyse_log_return_3 = log_return_data['nyse_log_return'].ix[i-2]
        djia_log_return_1 = log_return_data['djia_log_return'].ix[i]
        djia_log_return_2 = log_return_data['djia_log_return'].ix[i-1]
        djia_log_return_3 = log_return_data['djia_log_return'].ix[i-2]
        nikkei_log_return_1 = log_return_data['nikkei_log_return'].ix[i]
        nikkei_log_return_2 = log_return_data['nikkei_log_return'].ix[i-1]
        nikkei_log_return_3 = log_return_data['nikkei_log_return'].ix[i-2]
        hangseng_log_return_1 = log_return_data['hangseng_log_return'].ix[i]
        hangseng_log_return_2 = log_return_data['hangseng_log_return'].ix[i-1]
        hangseng_log_return_3 = log_return_data['hangseng_log_return'].ix[i-2]
        ftse_log_return_1 = log_return_data['ftse_log_return'].ix[i]
        ftse_log_return_2 = log_return_data['ftse_log_return'].ix[i-1]
        ftse_log_return_3 = log_return_data['ftse_log_return'].ix[i-2]
        dax_log_return_1 = log_return_data['dax_log_return'].ix[i]
        dax_log_return_2 = log_return_data['dax_log_return'].ix[i-1]
        dax_log_return_3 = log_return_data['dax_log_return'].ix[i-2]
        aord_log_return_1 = log_return_data['aord_log_return'].ix[i]
        aord_log_return_2 = log_return_data['aord_log_return'].ix[i-1]
        aord_log_return_3 = log_return_data['aord_log_return'].ix[i-2]

        # 各データをインデックスのラベルを使用しないで結合する。
        training_test_data = training_test_data.append(
            {
            'snp_log_return_positive':snp_log_return_positive,
            'snp_log_return_negative':snp_log_return_negative,
            'snp_log_return_1':snp_log_return_1,
            'snp_log_return_2':snp_log_return_2,
            'snp_log_return_3':snp_log_return_3,
            'nyse_log_return_1':nyse_log_return_1,
            'nyse_log_return_2':nyse_log_return_2,
            'nyse_log_return_3':nyse_log_return_3,
            'djia_log_return_1':djia_log_return_1,
            'djia_log_return_2':djia_log_return_2,
            'djia_log_return_3':djia_log_return_3,
            'nikkei_log_return_1':nikkei_log_return_1,
            'nikkei_log_return_2':nikkei_log_return_2,
            'nikkei_log_return_3':nikkei_log_return_3,
            'hangseng_log_return_1':hangseng_log_return_1,
            'hangseng_log_return_2':hangseng_log_return_2,
            'hangseng_log_return_3':hangseng_log_return_3,
            'ftse_log_return_1':ftse_log_return_1,
            'ftse_log_return_2':ftse_log_return_2,
            'ftse_log_return_3':ftse_log_return_3,
            'dax_log_return_1':dax_log_return_1,
            'dax_log_return_2':dax_log_return_2,
            'dax_log_return_3':dax_log_return_3,
            'aord_log_return_1':aord_log_return_1,
            'aord_log_return_2':aord_log_return_2,
            'aord_log_return_3':aord_log_return_3},
            ignore_index=True)
    
    # 3列目以降を説明変数として格納する。
    predictors_tf = training_test_data[training_test_data.columns[2:]]

    # 1、2列目を目的変数として格納する。
    classes_tf = training_test_data[training_test_data.columns[:2]] 

    # 説明変数の初めの80%を学習用データにする。
    x_train = predictors_tf.as_matrix()
    
    # 目的変数の初めの80%を学習用データにする。
    y_train = classes_tf.as_matrix()
    
    return x_train, y_train, nikkei.index.T

In [137]:
# modelの読み込み
# models.set_model_pkl('{0}.pkl'.format(target))
# models.set_model_name('{0}.model'.format(target))
# models.set_optimizer_name('{0}.state'.format(target))
# model=models.load_model()

# ヒストリカルデータの開始日と終了日を設定する。
start = datetime(2016, 11, 10)
end = datetime(2016, 11, 24)
nikkei = web.DataReader('^N225', 'yahoo', start, end)

x, y, index = set_production_data(start,end)
# print len(x)
h1 = F.dropout(F.relu(model.l1(Variable(x.astype(np.float32)))),  train=False)
h2 = F.dropout(F.relu(model.l2(h1)), train=False)
p  = model.l3(h2)



In [138]:
# print np.argmax(p.data)
for i, ans in enumerate(p.data):
    print ans, y[i], index[i]
print np.argmax(p.data)

[ 0.48943156  0.50108069] [ 1.  0.] 2016-11-10 00:00:00
[ 0.49722451  0.49589059] [ 1.  0.] 2016-11-11 00:00:00
[ 0.48279443  0.50594825] [ 1.  0.] 2016-11-14 00:00:00
5
